In [ ]:
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
import geopandas as gpd
import os
from tqdm import tqdm

In [16]:
def find_jp2_files_recursive(root_dir):
    jp2_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith(".tif"):
                full_path = os.path.join(dirpath, filename)
                jp2_files.append(full_path)
    return jp2_files

In [17]:
def merge_and_clip_jp2s(jp2_paths, shapefile_path, output_path):
    # Open all jp2 files with tqdm progress bar
    print("Opening JP2 files...")
    src_files_to_mosaic = [rasterio.open(fp) for fp in tqdm(jp2_paths, desc="Reading JP2s")]

    # Merge into a single raster
    print("Merging rasters...")
    mosaic, mosaic_transform = merge(src_files_to_mosaic)

    # Copy metadata from one of the source files and update it for the mosaic
    mosaic_meta = src_files_to_mosaic[0].meta.copy()
    mosaic_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": mosaic_transform,
        "count": mosaic.shape[0]
    })

    # Load shapefile and get geometry
    print("Reading shapefile...")
    shape = gpd.read_file(shapefile_path)
    shape = shape.to_crs(src_files_to_mosaic[0].crs)  # Match CRS
    geometry = shape.geometry.values

    # Clip the mosaic
    print("Clipping mosaic...")
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**mosaic_meta) as dataset:
            dataset.write(mosaic)
            clipped, clipped_transform = mask(dataset=dataset, shapes=geometry, crop=True)

    # Update metadata for clipped output
    clipped_meta = mosaic_meta.copy()
    clipped_meta.update({
        "height": clipped.shape[1],
        "width": clipped.shape[2],
        "transform": clipped_transform
    })

    # Write the clipped image to file
    print("Writing output raster...")
    with rasterio.open(output_path, "w", **clipped_meta) as dest:
        dest.write(clipped)

    print(f"Saved clipped raster to: {output_path}")

In [18]:
jp2_list = find_jp2_files_recursive("D:/terrain_generation_project/NAIP/")
clip_shapefile = "D:/terrain_generation_project/tif_extent/tif_extent.shp"
output_tif = "D:/terrain_generation_project/NAIP_processed/naip_study_area.tif"

In [19]:
print(jp2_list)

['D:/terrain_generation_project/NAIP/m_3411855_ne_11_060_20220505\\m_3411855_ne_11_060_20220505.tif', 'D:/terrain_generation_project/NAIP/m_3411855_nw_11_060_20220505\\m_3411855_nw_11_060_20220505.tif', 'D:/terrain_generation_project/NAIP/m_3411855_se_11_060_20220511\\m_3411855_se_11_060_20220511.tif', 'D:/terrain_generation_project/NAIP/m_3411855_sw_11_060_20220511\\m_3411855_sw_11_060_20220511.tif', 'D:/terrain_generation_project/NAIP/m_3411856_ne_11_060_20220507\\m_3411856_ne_11_060_20220507.tif', 'D:/terrain_generation_project/NAIP/m_3411856_nw_11_060_20220505\\m_3411856_nw_11_060_20220505.tif', 'D:/terrain_generation_project/NAIP/m_3411856_se_11_060_20220512\\m_3411856_se_11_060_20220512.tif', 'D:/terrain_generation_project/NAIP/m_3411856_sw_11_060_20220512\\m_3411856_sw_11_060_20220512.tif']


In [20]:
merge_and_clip_jp2s(jp2_list, clip_shapefile, output_tif)

Opening JP2 files...


TypeError: 'module' object is not callable